<a href="https://colab.research.google.com/github/JienWeng/stock-markets-analytics-zoomcamp_2025/blob/main/module2_sma_zoomboot_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Q1

In [27]:
import pandas as pd
import requests
import numpy as np
import yfinance as yf

In [10]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

url = "https://stockanalysis.com/ipos/withdrawn/"
response = requests.get(url, headers=headers)

df = pd.read_html(response.text)[0]

/tmp/ipython-input-10-3189484136.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(response.text)[0]


In [12]:
df

,Symbol,Company Name,Price Range,Shares Offered
0,ODTX,"Odyssey Therapeutics, Inc.",-,-
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000
2,AURN,"Aurion Biotech, Inc.",-,-
3,ROTR,"PHI Group, Inc.",-,-
4,ONE,One Power Company,-,-
...,...,...,...,...
95,FHP,"Freehold Properties, Inc.",-,-
96,CHO,Chobani Inc.,-,-
97,IFIT,iFIT Health & Fitness Inc.,$18.00 - $21.00,30769231
98,GLGX,"Gerson Lehrman Group, Inc.",-,-


In [15]:
import re

def classify_company(name):
    name = name.lower()

    if re.search(r"\bacquisition (corp|corporation)\b", name):
        return "Acq.Corp"
    elif re.search(r"\b(inc|incorporated)\b", name):
        return "Inc"
    elif re.search(r"\bgroup\b", name):
        return "Group"
    elif re.search(r"\b(ltd|limited)\b", name):
        return "Limited"
    elif re.search(r"\bholdings\b", name):
        return "Holdings"
    else:
        return "Other"

df["Company Class"] = df["Company Name"].apply(classify_company)

In [16]:
df

,Symbol,Company Name,Price Range,Shares Offered,Company Class
0,ODTX,"Odyssey Therapeutics, Inc.",-,-,Inc
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000,Inc
2,AURN,"Aurion Biotech, Inc.",-,-,Inc
3,ROTR,"PHI Group, Inc.",-,-,Inc
4,ONE,One Power Company,-,-,Other
...,...,...,...,...,...
95,FHP,"Freehold Properties, Inc.",-,-,Inc
96,CHO,Chobani Inc.,-,-,Inc
97,IFIT,iFIT Health & Fitness Inc.,$18.00 - $21.00,30769231,Inc
98,GLGX,"Gerson Lehrman Group, Inc.",-,-,Inc


In [18]:
def parse_avg_price(price_range):
    if isinstance(price_range, str):
        if '-' in price_range:
            parts = price_range.replace('$', '').split('-')
            try:
                return (float(parts[0]) + float(parts[1])) / 2
            except:
                return None
        else:
            try:
                return float(price_range.replace('$', ''))
            except:
                return None
    return None

df["Avg. Price"] = df["Price Range"].apply(parse_avg_price)

In [19]:
df

,Symbol,Company Name,Price Range,Shares Offered,Company Class,Avg. Price
0,ODTX,"Odyssey Therapeutics, Inc.",-,-,Inc,NaN
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000,Inc,3.5
2,AURN,"Aurion Biotech, Inc.",-,-,Inc,NaN
3,ROTR,"PHI Group, Inc.",-,-,Inc,NaN
4,ONE,One Power Company,-,-,Other,NaN
...,...,...,...,...,...,...
95,FHP,"Freehold Properties, Inc.",-,-,Inc,NaN
96,CHO,Chobani Inc.,-,-,Inc,NaN
97,IFIT,iFIT Health & Fitness Inc.,$18.00 - $21.00,30769231,Inc,19.5
98,GLGX,"Gerson Lehrman Group, Inc.",-,-,Inc,NaN


In [20]:
df["Shares Offered"] = pd.to_numeric(df["Shares Offered"].str.replace(',', ''), errors='coerce')

In [21]:
df["Withdrawn Value"] = df["Shares Offered"] * df["Avg. Price"]

In [22]:
grouped = df.groupby("Company Class", dropna=False)["Withdrawn Value"].sum().sort_values(ascending=False)
print(grouped)

Company Class
Acq.Corp    4.021000e+09
Inc         2.257164e+09
Other       7.679200e+08
Limited     5.497346e+08
Holdings    7.500000e+07
Group       3.378750e+07
Name: Withdrawn Value, dtype: float64


# Q2

In [23]:
url = "https://stockanalysis.com/ipos/2024/"
response = requests.get(url, headers=headers)
ipos_2024 = pd.read_html(response.text)[0]

/tmp/ipython-input-23-3734048503.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  ipos_2024 = pd.read_html(response.text)[0]


In [24]:
ipos_2024["IPO Date"] = pd.to_datetime(ipos_2024["IPO Date"])
filtered_ipos = ipos_2024[ipos_2024["IPO Date"] < "2024-06-01"].copy()

In [26]:
tickers = filtered_ipos["Symbol"].dropna().unique().tolist()

In [28]:
data = yf.download(tickers, start="2024-01-01", end="2025-06-07", group_by='ticker', auto_adjust=True)


[*********************100%***********************]  77 of 77 completed


In [29]:
data

Ticker             AS                                              MNDR  \
Price            Open       High        Low      Close     Volume  Open   
Date                                                                      
2024-01-09        NaN        NaN        NaN        NaN        NaN   NaN   
2024-01-10        NaN        NaN        NaN        NaN        NaN   NaN   
2024-01-11        NaN        NaN        NaN        NaN        NaN   NaN   
2024-01-12        NaN        NaN        NaN        NaN        NaN   NaN   
2024-01-16        NaN        NaN        NaN        NaN        NaN   NaN   
...               ...        ...        ...        ...        ...   ...   
2025-06-02  36.430000  36.840000  36.110001  36.810001  7603800.0  1.38   
2025-06-03  36.849998  37.509998  36.570000  37.459999  6634400.0  1.36   
2025-06-04  37.720001  37.770000  37.040001  37.230000  4236900.0  1.34   
2025-06-05  37.080002  38.110001  37.060001  37.380001  4280000.0  1.36   
2025-06-06  37.540001  37.799999  37.009998  37.520000  4036300.0  1.31   

Ticker                                    ...   MAMO                      \
Price        High    Low Close    Volume  ...   Open  High    Low  Close   
Date                                      ...                              
2024-01-09    NaN    NaN   NaN       NaN  ...    NaN   NaN    NaN    NaN   
2024-01-10    NaN    NaN   NaN       NaN  ...    NaN   NaN    NaN    NaN   
2024-01-11    NaN    NaN   NaN       NaN  ...    NaN   NaN    NaN    NaN   
2024-01-12    NaN    NaN   NaN       NaN  ...    NaN   NaN    NaN    NaN   
2024-01-16    NaN    NaN   NaN       NaN  ...    NaN   NaN    NaN    NaN   
...           ...    ...   ...       ...  ...    ...   ...    ...    ...   
2025-06-02  1.420  1.340  1.39   37900.0  ...  2.350  2.35  2.200  2.205   
2025-06-03  1.390  1.350  1.36   19500.0  ...  2.221  2.26  2.145  2.223   
2025-06-04  1.383  1.311  1.32   39800.0  ...  2.060  2.23  2.060  2.100   
2025-06-05  1.370  1.300  1.33  301500.0  ...  2.070  2.10  2.035  2.099   
2025-06-06  1.350  1.300  1.34   51800.0  ...  2.000  2.20  1.848  2.100   

Ticker                 MTEN                                   
Price        Volume    Open    High     Low   Close   Volume  
Date                                                          
2024-01-09      NaN     NaN     NaN     NaN     NaN      NaN  
2024-01-10      NaN     NaN     NaN     NaN     NaN      NaN  
2024-01-11      NaN     NaN     NaN     NaN     NaN      NaN  
2024-01-12      NaN     NaN     NaN     NaN     NaN      NaN  
2024-01-16      NaN     NaN     NaN     NaN     NaN      NaN  
...             ...     ...     ...     ...     ...      ...  
2025-06-02   2500.0  11.700  13.495  11.445  12.272  34900.0  
2025-06-03   5300.0  12.100  12.500  11.700  12.182  31900.0  
2025-06-04  16500.0  12.295  12.600  11.850  12.355  44800.0  
2025-06-05   3900.0  12.010  12.095  11.400  11.885  32200.0  
2025-06-06   9100.0  11.810  12.300  11.810  12.290  41800.0  

[354 rows x 385 columns]

In [31]:
results = []

for ticker in tickers:
    try:
        df = data[ticker].copy()
        df = df.dropna()
        df["growth_252d"] = df["Close"] / df["Close"].shift(252)
        df["volatility"] = df["Close"].rolling(30).std() * np.sqrt(252)
        df["Sharpe"] = (df["growth_252d"] - 0.045) / df["volatility"]
        df["Ticker"] = ticker
        results.append(df)
    except Exception:
        continue

stocks_df = pd.concat(results)

In [32]:
stocks_df

Price,Open,High,Low,Close,Volume,growth_252d,volatility,Sharpe,Ticker
Date,,,,,,,,,
2024-05-31,4.00,4.200,2.800,3.02,440600.0,NaN,NaN,NaN,NAKA
2024-06-03,2.99,3.110,2.350,2.66,147300.0,NaN,NaN,NaN,NAKA
2024-06-04,2.53,3.110,2.410,2.92,73800.0,NaN,NaN,NaN,NAKA
2024-06-05,2.91,3.090,2.600,2.73,51100.0,NaN,NaN,NaN,NAKA
2024-06-06,2.94,2.940,2.410,2.69,56500.0,NaN,NaN,NaN,NAKA
...,...,...,...,...,...,...,...,...,...
2025-06-02,3.10,3.480,2.940,3.36,511000.0,5.014925,10.446051,0.475771,ROMA
2025-06-03,3.36,3.530,3.160,3.41,67200.0,5.262346,10.986046,0.474907,ROMA
2025-06-04,3.50,3.850,3.457,3.76,271100.0,5.829458,11.789832,0.490631,ROMA


In [33]:
stocks_df = stocks_df.reset_index()
stocks_df = stocks_df[stocks_df["Date"] == "2025-06-06"]

In [34]:
stocks_df[["growth_252d", "Sharpe"]].describe()

Price,growth_252d,Sharpe
count,73.000000,73.000000
mean,1.227948,0.284576
std,1.480237,0.512601
min,0.024970,-0.079677
25%,0.293510,0.040265
50%,0.763188,0.083768
75%,1.446667,0.291048
max,8.097413,2.835668


# Q3

In [44]:
headers = {"User-Agent": "Mozilla/5.0"}
url = "https://stockanalysis.com/ipos/2024/"
response = requests.get(url, headers=headers)
ipos_2024 = pd.read_html(response.text)[0]
ipos_2024["IPO Date"] = pd.to_datetime(ipos_2024["IPO Date"])
filtered_ipos = ipos_2024[ipos_2024["IPO Date"] < "2024-06-01"]
tickers = filtered_ipos["Symbol"].dropna().unique().tolist()

data = yf.download(tickers, start="2024-01-01", end="2025-06-21", group_by="ticker", auto_adjust=True)

/tmp/ipython-input-44-519189111.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  ipos_2024 = pd.read_html(response.text)[0]
[*********************100%***********************]  77 of 77 completed


In [45]:
data

Ticker             AS                                            MNDR        \
Price            Open       High     Low      Close     Volume   Open  High   
Date                                                                          
2024-01-09        NaN        NaN     NaN        NaN        NaN    NaN   NaN   
2024-01-10        NaN        NaN     NaN        NaN        NaN    NaN   NaN   
2024-01-11        NaN        NaN     NaN        NaN        NaN    NaN   NaN   
2024-01-12        NaN        NaN     NaN        NaN        NaN    NaN   NaN   
2024-01-16        NaN        NaN     NaN        NaN        NaN    NaN   NaN   
...               ...        ...     ...        ...        ...    ...   ...   
2025-06-13  36.919998  37.230000  36.020  36.240002  4981100.0  1.250  1.26   
2025-06-16  36.750000  37.529999  36.750  37.119999  3263500.0  1.249  1.25   
2025-06-17  36.799999  36.869999  35.930  36.060001  3124600.0  1.220  1.24   
2025-06-18  36.000000  36.799999  35.910  36.189999  2548900.0  1.230  1.59   
2025-06-20  36.360001  36.650002  36.013  36.419998  3333300.0  1.400  1.41   

Ticker                               ...   MTEN                        \
Price         Low  Close     Volume  ...   Open    High    Low  Close   
Date                                 ...                                
2024-01-09    NaN    NaN        NaN  ...    NaN     NaN    NaN    NaN   
2024-01-10    NaN    NaN        NaN  ...    NaN     NaN    NaN    NaN   
2024-01-11    NaN    NaN        NaN  ...    NaN     NaN    NaN    NaN   
2024-01-12    NaN    NaN        NaN  ...    NaN     NaN    NaN    NaN   
2024-01-16    NaN    NaN        NaN  ...    NaN     NaN    NaN    NaN   
...           ...    ...        ...  ...    ...     ...    ...    ...   
2025-06-13  1.210  1.220    60500.0  ...  12.01  12.325  11.67  12.08   
2025-06-16  1.200  1.220    46600.0  ...  12.00  13.200  11.88  12.22   
2025-06-17  1.212  1.229    18600.0  ...  12.22  12.310  11.85  12.18   
2025-06-18  1.202  1.389  2025900.0  ...  12.08  12.350  10.80  12.09   
2025-06-20  1.274  1.330   414100.0  ...  12.11  12.430  11.63  12.15   

Ticker                MAMO                               
Price        Volume   Open   High    Low  Close  Volume  
Date                                                     
2024-01-09      NaN    NaN    NaN    NaN    NaN     NaN  
2024-01-10      NaN    NaN    NaN    NaN    NaN     NaN  
2024-01-11      NaN    NaN    NaN    NaN    NaN     NaN  
2024-01-12      NaN    NaN    NaN    NaN    NaN     NaN  
2024-01-16      NaN    NaN    NaN    NaN    NaN     NaN  
...             ...    ...    ...    ...    ...     ...  
2025-06-13  38800.0  2.170  2.170  2.000  2.084  1500.0  
2025-06-16  65700.0  1.883  2.187  1.839  2.100  1400.0  
2025-06-17  68900.0  2.110  2.120  2.060  2.060  1100.0  
2025-06-18  45700.0  2.130  2.148  2.023  2.050  1900.0  
2025-06-20  38900.0  2.200  2.200  1.970  1.970  1100.0  

[363 rows x 385 columns]

In [46]:
results = []
for ticker in tickers:
    try:
        df = data[ticker].copy()
        df = df.dropna()
        df["Ticker"] = ticker
        df = df.reset_index()
        for month in range(1, 13):
            days = 21 * month
            df[f"future_growth_{month}m"] = df["Close"].shift(-days) / df["Close"]
        results.append(df)
    except Exception:
        continue

future_growth_df = pd.concat(results, ignore_index=True)


In [47]:
future_growth_df

Price,Date,Open,High,Low,Close,Volume,Ticker,future_growth_1m,future_growth_2m,future_growth_3m,future_growth_4m,future_growth_5m,future_growth_6m,future_growth_7m,future_growth_8m,future_growth_9m,future_growth_10m,future_growth_11m,future_growth_12m
0,2024-05-31,4.000,4.200,2.80,3.020,440600.0,NAKA,0.728477,0.552980,0.397351,0.350993,0.341060,0.387417,0.410596,0.397351,0.586093,0.516556,0.897351,5.672185
1,2024-06-03,2.990,3.110,2.35,2.660,147300.0,NAKA,0.924812,0.609023,0.439850,0.390226,0.379699,0.428571,0.529323,0.454887,0.642857,0.556391,1.150376,6.022556
2,2024-06-04,2.530,3.110,2.41,2.920,73800.0,NAKA,0.770548,0.518836,0.434931,0.349315,0.332192,0.541096,0.523973,0.414384,0.565068,0.489726,1.058219,5.438356
3,2024-06-05,2.910,3.090,2.60,2.730,51100.0,NAKA,0.824176,0.560440,0.487179,0.378388,0.340659,0.648352,0.511722,0.449084,0.564103,0.597070,1.384615,4.890110
4,2024-06-06,2.940,2.940,2.41,2.690,56500.0,NAKA,0.910781,0.555390,0.496283,0.405204,0.334572,0.620818,0.609665,0.464684,0.527881,0.650558,1.449814,4.312268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23875,2025-06-13,2.870,2.890,2.56,2.660,123100.0,ROMA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23876,2025-06-16,2.840,3.000,2.64,2.875,63100.0,ROMA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23877,2025-06-17,2.850,2.935,2.79,2.795,10200.0,ROMA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23878,2025-06-18,2.883,2.900,2.73,2.790,33600.0,ROMA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
min_date_df = future_growth_df.groupby("Ticker")["Date"].min().reset_index()
min_date_df.rename(columns={"Date": "min_date"}, inplace=True)


In [49]:
future_growth_df["Date"] = pd.to_datetime(future_growth_df["Date"])
joined = pd.merge(
    min_date_df,
    future_growth_df,
    left_on=["Ticker", "min_date"],
    right_on=["Ticker", "Date"],
    how="inner"
)

future_growth_df

Price,Date,Open,High,Low,Close,Volume,Ticker,future_growth_1m,future_growth_2m,future_growth_3m,future_growth_4m,future_growth_5m,future_growth_6m,future_growth_7m,future_growth_8m,future_growth_9m,future_growth_10m,future_growth_11m,future_growth_12m
0,2024-05-31,4.000,4.200,2.80,3.020,440600.0,NAKA,0.728477,0.552980,0.397351,0.350993,0.341060,0.387417,0.410596,0.397351,0.586093,0.516556,0.897351,5.672185
1,2024-06-03,2.990,3.110,2.35,2.660,147300.0,NAKA,0.924812,0.609023,0.439850,0.390226,0.379699,0.428571,0.529323,0.454887,0.642857,0.556391,1.150376,6.022556
2,2024-06-04,2.530,3.110,2.41,2.920,73800.0,NAKA,0.770548,0.518836,0.434931,0.349315,0.332192,0.541096,0.523973,0.414384,0.565068,0.489726,1.058219,5.438356
3,2024-06-05,2.910,3.090,2.60,2.730,51100.0,NAKA,0.824176,0.560440,0.487179,0.378388,0.340659,0.648352,0.511722,0.449084,0.564103,0.597070,1.384615,4.890110
4,2024-06-06,2.940,2.940,2.41,2.690,56500.0,NAKA,0.910781,0.555390,0.496283,0.405204,0.334572,0.620818,0.609665,0.464684,0.527881,0.650558,1.449814,4.312268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23875,2025-06-13,2.870,2.890,2.56,2.660,123100.0,ROMA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23876,2025-06-16,2.840,3.000,2.64,2.875,63100.0,ROMA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23877,2025-06-17,2.850,2.935,2.79,2.795,10200.0,ROMA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23878,2025-06-18,2.883,2.900,2.73,2.790,33600.0,ROMA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
stats = joined[[f"future_growth_{m}m" for m in range(1, 13)]].describe()


In [51]:
avg_growths = joined[[f"future_growth_{m}m" for m in range(1, 13)]].mean()
optimal_month = avg_growths.idxmax()
int(optimal_month.split('_')[2][0:-1])

12

# Q4

In [52]:
import gdown
import pandas as pd

file_id = "1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-"
gdown.download(f"https://drive.google.com/uc?id={file_id}", "data.parquet", quiet=False)
df = pd.read_parquet("data.parquet", engine="pyarrow")


Downloading...
From (original): https://drive.google.com/uc?id=1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-
From (redirected): https://drive.google.com/uc?id=1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-&confirm=t&uuid=181b91f7-64a3-4725-a4a1-c30026ee39eb
To: /content/data.parquet
100%|██████████| 130M/130M [00:02<00:00, 52.8MB/s]


In [53]:
rsi_threshold = 25
selected_df = df[
    (df['rsi'] < rsi_threshold) &
    (df['Date'] >= '2000-01-01') &
    (df['Date'] <= '2025-06-01')
]

In [54]:
net_income = 1000 * (selected_df['growth_future_30d'] - 1).sum()

In [55]:
net_income

np.float64(24295.523125248386)